In [143]:
from train_functions import train_sahp
import torch
import pickle
import numpy as np

from utils.load_synth_data import process_loaded_sequences
from utils.util import get_batch

from train_functions.train_sahp import make_model,eval_sahp,prediction_evaluation,MaskBatch
import seaborn as sns

## Synth Model

In [2]:
model = train_sahp.make_model(max_sequence_length=324)
model_dict =torch.load('saved_models/sahp-synthetic_hidden16-20210622-205430',map_location=torch.device('cpu'))
model.load_state_dict(model_dict)

<All keys matched successfully>

In [12]:
## Load Data Set
with open('data/simulated/hawkes_synthetic_random_2d_20191130-180837.pkl', 'rb') as f:
    loaded_hawkes_data = pickle.load(f)

tmax = loaded_hawkes_data['tmax']

seq_times, seq_types, seq_lengths, _ = process_loaded_sequences(loaded_hawkes_data, 2)

total_sample_size = seq_times.size(0)
train_ratio = 0.8
train_size = int(train_ratio * total_sample_size)
dev_ratio =0.1
dev_size = int(dev_ratio * total_sample_size)
## Split Traning and Test Sets
train_seq_times = seq_times[:train_size]
train_seq_types = seq_types[:train_size]
train_seq_lengths = seq_lengths[:train_size]


dev_seq_times = seq_times[train_size:train_size + dev_size]  # train_size+dev_size
dev_seq_types = seq_types[train_size:train_size + dev_size]
dev_seq_lengths = seq_lengths[train_size:train_size + dev_size]

test_seq_times = seq_times[-dev_size:]
test_seq_types = seq_types[-dev_size:]
test_seq_lengths = seq_lengths[-dev_size:]


## sequence length
train_seq_lengths, reorder_indices_train = train_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
train_seq_times = train_seq_times[reorder_indices_train]
train_seq_types = train_seq_types[reorder_indices_train]
#
dev_seq_lengths, reorder_indices_dev = dev_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
dev_seq_times = dev_seq_times[reorder_indices_dev]
dev_seq_types = dev_seq_types[reorder_indices_dev]

test_seq_lengths, reorder_indices_test = test_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
test_seq_times = test_seq_times[reorder_indices_test]
test_seq_types = test_seq_types[reorder_indices_test]

max_sequence_length = max(train_seq_lengths[0], dev_seq_lengths[0], test_seq_lengths[0])
print('max_sequence_length: {}'.format(max_sequence_length))

max_sequence_length: 323


In [50]:
torch.manual_seed(42)
batch_size = 32
test_size = test_seq_times.size(0)
device = 'cpu'
test_loop_range = list(range(0, test_size, batch_size))

## Get test Loss
test_event_num, epoch_test_loss = eval_sahp(batch_size, test_loop_range, test_seq_lengths, test_seq_times,
                                            test_seq_types, model, device, 0)

test_loss = epoch_test_loss/test_event_num

## Get Result Metrics
avg_rmse, types_predict_score, results = prediction_evaluation(
    device, model, test_seq_lengths, test_seq_times, test_seq_types, test_size, tmax)

print(test_loss)


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:16<00:00, 23.93it/s]

rmse 17.319760118898063
Type prediction score: 0.5575
tensor(0.6343)


In [133]:
incr_errors = ((incr_estimates - incr_reals) / +incr_reals) ** 2 
avg_rmse = np.sqrt(np.mean(incr_errors), dtype=np.float64)
avg_mae = np.mean(np.abs(incr_estimates - incr_reals))

## Get Intensities

In [162]:
# dt_seq = test_seq_times[:, 1:] - test_seq_times[:, :-1]

i_batch = 0 
batch_onehot, batch_seq_times, batch_dt, batch_seq_types, _, _, _, batch_seq_lengths = \
get_batch(batch_size, i_batch, model, test_seq_lengths, test_seq_times, test_seq_types, rnn=False)

batch_seq_types = batch_seq_types[:, 1:]

masked_seq_types = MaskBatch(batch_seq_types, pad=model.process_dim,
                             device=device)  # exclude the first added event
model.forward(batch_dt, masked_seq_types.src, masked_seq_types.src_mask)



cell_t = model.state_decay(model.converge_point, model.start_point, model.omega, batch_dt[:, :, None])
intens_at_evs = model.intensity_layer(cell_t)

In [173]:
batch_dt[:, :, None][0]

tensor([[8.1986e+00],
        [3.9271e+00],
        [1.8315e+00],
        [3.4096e-01],
        [9.1432e-02],
        [2.0610e+00],
        [2.0708e+00],
        [2.3338e+00],
        [6.3303e-02],
        [1.4945e-01],
        [9.4750e-01],
        [6.6881e-01],
        [1.1348e-01],
        [2.6611e-02],
        [2.5726e-01],
        [1.0474e+00],
        [1.0061e-01],
        [2.0206e-02],
        [4.6667e-01],
        [4.1332e-01],
        [5.8114e-01],
        [1.1904e-01],
        [2.0031e-02],
        [3.8305e-01],
        [6.3133e-04],
        [5.1162e-01],
        [1.4303e-01],
        [6.2770e-01],
        [1.2775e-01],
        [8.3595e-01],
        [6.0497e-01],
        [1.7657e+00],
        [6.0880e-01],
        [9.3099e-01],
        [2.5266e-01],
        [8.9049e-01],
        [6.0435e-01],
        [3.6918e-01],
        [5.2655e-01],
        [3.2891e-01],
        [1.1824e-01],
        [1.8523e-01],
        [2.5787e-01],
        [1.7093e-01],
        [4.5868e-01],
        [5

In [169]:
intens_at_evs

tensor([[[0.0376, 0.2483],
         [0.0593, 0.6184],
         [0.0655, 0.8383],
         ...,
         [1.4067, 0.0671],
         [0.2394, 0.0313],
         [0.5005, 0.0171]],

        [[0.9951, 0.0237],
         [0.2292, 2.3567],
         [2.0243, 0.1277],
         ...,
         [0.0853, 1.3355],
         [0.0478, 0.4635],
         [0.2000, 0.0313]],

        [[0.2219, 0.0520],
         [1.5166, 0.1330],
         [0.4549, 0.0347],
         ...,
         [0.0927, 0.0784],
         [0.0538, 0.2799],
         [0.1784, 0.0380]],

        ...,

        [[0.0329, 0.4793],
         [0.1229, 1.6493],
         [0.0709, 1.1362],
         ...,
         [0.0961, 0.0729],
         [0.0706, 0.1087],
         [0.1616, 0.0495]],

        [[0.9629, 0.2178],
         [1.4417, 0.3399],
         [0.2545, 2.3661],
         ...,
         [0.0600, 0.1382],
         [0.1440, 0.0408],
         [0.1970, 0.0317]],

        [[0.0476, 0.4474],
         [0.7276, 0.0391],
         [0.0295, 0.2966],
         ...,
 

In [167]:
batch_dt[:, :, None]

tensor([[[8.1986],
         [3.9271],
         [1.8315],
         ...,
         [0.6408],
         [2.1762],
         [0.2232]],

        [[2.8910],
         [0.0323],
         [0.0513],
         ...,
         [0.2090],
         [1.3554],
         [0.7401]],

        [[2.4071],
         [0.0458],
         [1.9676],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        ...,

        [[0.7240],
         [2.5218],
         [1.0942],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.3943],
         [1.1565],
         [0.0341],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[1.7820],
         [1.4197],
         [5.4799],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]]])